In [11]:
## header file function (need to be script )
import sys
import numpy as np
import vireoSNP
from vireoSNP.utils.vireo_base import match
from vireoSNP.utils.vcf_utils import load_VCF, write_VCF, parse_donor_GPb
from vireoSNP.utils.vcf_utils import read_sparse_GeneINFO, GenoINFO_maker
import matplotlib.pyplot as plt
import pandas as pd
from vireoSNP.utils.io_utils import match_donor_VCF
import re as re
def VCF_to_sdf(bulk_data):
    df_empty=pd.DataFrame(columns=["variants","AD","DP","GT"])
    bulk_dat = read_sparse_GeneINFO(bulk_data['GenoINFO'], keys=['AD', 'DP'])
    df_empty["variants"]=bulk_data["variants"]
    df_empty["AD"]=bulk_dat["AD"].toarray()
    df_empty["DP"]=bulk_dat["DP"].toarray()
    return (df_empty)

def rematch_vcf(bulk_data,donor_vcf):
    df_bulk=VCF_to_sdf(bulk_data)
    df_donor=pd.DataFrame(columns=["variants","GT"])
    df_donor["variants"]=donor_vcf["variants"]
    df_donor["GT"]=donor_vcf["GenoINFO"]["GT"]     
    donor_tensor = vireoSNP.vcf.parse_donor_GPb(donor_vcf['GenoINFO']["GT"], "GT")
    itersect_variants=pd.Series(list(set(df_donor["variants"]).intersection(set(df_bulk["variants"]))))
    id_d=df_donor["variants"].isin(itersect_variants)
    id_b=df_bulk["variants"].isin(itersect_variants)
    df_bulk=df_bulk[id_b]
    df_donor=df_donor[id_d]
    donor_tensor=donor_tensor[id_d]
    return(df_bulk,df_donor,donor_tensor)
def rematch_vcf_annotated(bulk_data,donor_vcf,donor_vcf_anno):
    df_bulk=VCF_to_sdf(bulk_data)
    df_donora=pd.DataFrame(columns=["variants","function","genes","avsnp","GT"])
    df_donora["variants"]=donor_vcf_anno["variants"]  
    df_donora["function"]=donor_vcf_anno["function."]
    df_donora["genes"]=donor_vcf_anno["genes"]
    df_donora["avsnp"]=donor_vcf_anno["avsnp"]
    df_donor=pd.DataFrame(columns=["variants","GT"])
    df_donor["variants"]=donor_vcf["variants"]
    df_donor["GT"]=donor_vcf["GenoINFO"]["GT"]     
    donor_tensor = vireoSNP.vcf.parse_donor_GPb(donor_vcf['GenoINFO']["GT"], "GT")
    itersect_variants=pd.Series(list(set(df_donora["variants"]).intersection(set(df_bulk["variants"]))))
    id_d=df_donor["variants"].isin(itersect_variants)
    id_b=df_bulk["variants"].isin(itersect_variants)
    id_da=df_donora["variants"].isin(itersect_variants)
    df_bulk=df_bulk[id_b]
    df_donor=df_donor[id_d]
    df_donora=df_donora[id_da]
    donor_tensor=donor_tensor[id_d]
    df_donora["GT"]=df_donora["GT"]=df_donor["GT"].tolist()
    return(df_bulk,df_donora,donor_tensor)
def merged_VCF_to_sdf(bulk_data_merge):
    df_empty=pd.DataFrame(columns=["variants","AD","DP","OTH"])
    df_empty["variants"]=bulk_data_merge["variants"]
    testdf=pd.DataFrame(np.array(bulk_merged_data['FixedINFO']["INFO"]))
    df_empty["AD"]=testdf[0].str.split(';',expand=True)[0].str.split('=',expand=True)[1].astype(float)
    df_empty["DP"]=testdf[0].str.split(';',expand=True)[1].str.split('=',expand=True)[1].astype(float)
    df_empty["OTH"]=testdf[0].str.split(';',expand=True)[2].str.split('=',expand=True)[1].astype(float)

    

    return(df_empty)
def rematch_merge_vcf_annotated(bulk_data,donor_vcf,donor_vcf_anno):
    df_bulk=merged_VCF_to_sdf(bulk_data)
    df_donora=pd.DataFrame(columns=["variants","function","genes","avsnp","GT"])
    df_donora["variants"]=donor_vcf_anno["variants"]  
    df_donora["function"]=donor_vcf_anno["function."]
    df_donora["genes"]=donor_vcf_anno["genes"]
    df_donora["avsnp"]=donor_vcf_anno["avsnp"]
    df_donor=pd.DataFrame(columns=["variants","GT"])
    df_donor["variants"]=donor_vcf["variants"]
    df_donor["GT"]=donor_vcf["GenoINFO"]["GT"]     
    donor_tensor = vireoSNP.vcf.parse_donor_GPb(donor_vcf['GenoINFO']["GT"], "GT")
    itersect_variants=pd.Series(list(set(df_donora["variants"]).intersection(set(df_bulk["variants"]))))
    id_d=df_donor["variants"].isin(itersect_variants)
    id_b=df_bulk["variants"].isin(itersect_variants)
    id_da=df_donora["variants"].isin(itersect_variants)
    df_bulk=df_bulk[id_b]
    df_donor=df_donor[id_d]
    df_donora=df_donora[id_da]
    donor_tensor=donor_tensor[id_d]
    df_donora["GT"]=df_donora["GT"]=df_donor["GT"].tolist()
    return(df_bulk,df_donora,donor_tensor)
def find_all(data, s):
    r_list = []
    for r in range(len(data)):
        r_list.append( s in data[r])
 
    return(r_list)
def removedfault(df_bulk,df_donora,donor_tensor):
    id_d=[]
    for bool in find_all(np.array(df_donora["GT"]).tolist(),"./."):
        id_d.append(not bool)
    df_donor=df_donora[id_d]
    df_gt=donor_tensor[id_d]
    id_b=df_bulk["variants"].isin(df_donor["variants"])
    df_b=df_bulk[id_b]
    return(df_b,df_donor,df_gt)

In [12]:
def prediction_bulk(df_b,df_d,df_gt,donor_vcf,pre_model,genelist):
    gene_unique=genelist
    new=pd.DataFrame(index=gene_unique,columns=donor_vcf['samples']+["chi","p","numSNP"])
    pre_psi=pre_model
    test_d=df_d
    test_b=df_b
    test_gt=df_gt
    sample_list=donor_vcf['samples']
    nk=len(donor_vcf['samples'])
    for i in range(len(gene_unique)):
        sub_d=test_d[test_d["genes"]==gene_unique[i]]
        sub_b=test_b[test_b["variants"].isin(test_d[test_d["genes"]==gene_unique[i]]["variants"])]
        sub_gt=test_gt[test_d["genes"]==gene_unique[i]]
        a=np.array(sub_b["AD"])
        d=np.array(sub_b["DP"])
        a=np.array(list(map(float,a)))
        d=np.array(list(map(float,d)))
        model =vireoSNP.VireoBulk(nk)
        model.fit(a,d,sub_gt,learn_theta=False )
        pv=vireoSNP.LikRatio_test(model.psi,pre_psi,a,d,sub_gt,model.theta)
        for j in range(len(model.psi)):
            new[sample_list[j]][i]=model.psi[j]
        new["chi"][i]=pv[0]
        new["p"][i]=pv[1]
        new["numSNP"][i]=len(a)
    return(new)

In [13]:
donor_vcf=load_VCF(   "/home/flyflyzhizhi/Vireobulk_analysis/data/filter_pbmc10donors.vcf.gz",    biallelic_only=True,    load_sample=True, sparse=False , format_list=None)
####count based model
##bulk_merged_data=load_VCF(   "/media/wgs/subset_stimulation_bam/pbmc10donor_bulk/cellSNP.cells.vcf.gz",    biallelic_only=True,    load_sample=True,    format_list=None,)
#### UMI based model
bulk_merged_data=load_VCF("/home/flyflyzhizhi/Vireobulk_analysis/data/cellSNP.base.vcf.gz", biallelic_only=True,load_sample=False,format_list=None,)
###processedcsv
df_anno=pd.read_csv("/home/flyflyzhizhi/Vireobulk_analysis/data/scvcf_annotated_processed.csv")

In [ ]:
bulk_merged_data.t

In [15]:
df_b

,variants,AD,DP,OTH
0,chr1_14574_A_G,2.0,35.0,1.0
1,chr1_14590_G_A,0.0,50.0,0.0
2,chr1_14599_T_A,0.0,53.0,2.0
3,chr1_14604_A_G,1.0,60.0,0.0
4,chr1_14610_T_C,1.0,62.0,1.0
...,...,...,...,...
128194,chrY_13390522_T_C,66.0,66.0,0.0
128195,chrY_19413646_C_T,0.0,20.0,0.0
128196,chrY_19413653_G_A,0.0,20.0,0.0
128197,chrY_19604311_A_T,45.0,45.0,0.0


In [14]:
%%time
df_b=merged_VCF_to_sdf(bulk_merged_data)

CPU times: user 3.34 s, sys: 7.52 ms, total: 3.34 s
Wall time: 3.34 s


In [32]:
df_b,df_d,m_GT=rematch_vcf(bulk_merged_data, donor_vcf)

KeyError: 'GenoINFO'

In [ ]:
#count_bulk_data
count_bulk_data=load_VCF(   "/media/wgs/subset_stimulation_bam/pbmc10donor_bulk/cellSNP.cells.vcf.gz",    biallelic_only=True,    load_sample=True,    format_list=None,)
cf_b,cf_d,cm_GT=rematch_merge_vcf_annotated(count_bulk_data,donor_vcf,df_anno)
cf_b,cf_d,cm_GT=removedfault(cf_b,cf_d,cm_GT)

In [30]:
donor_tensor = vireoSNP.vcf.parse_donor_GPb(donor_vcf['GenoINFO']["GT"], "GT")

In [31]:
donor_tensor

array([[[1.        , 0.        , 0.        ],
        [0.        , 1.        , 0.        ],
        [0.        , 1.        , 0.        ],
        ...,
        [1.        , 0.        , 0.        ],
        [0.        , 1.        , 0.        ],
        [0.        , 1.        , 0.        ]],

       [[1.        , 0.        , 0.        ],
        [0.        , 1.        , 0.        ],
        [0.        , 1.        , 0.        ],
        ...,
        [1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [0.        , 1.        , 0.        ]],

       [[0.        , 1.        , 0.        ],
        [0.        , 1.        , 0.        ],
        [0.        , 1.        , 0.        ],
        ...,
        [0.        , 1.        , 0.        ],
        [0.        , 1.        , 0.        ],
        [0.        , 1.        , 0.        ]],

       ...,

       [[0.        , 1.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0

In [ ]:
VCF_to_sdf(bulk_merged_data,)

In [33]:
%%time
df_b,df_d,m_GT=rematch_merge_vcf_annotated(bulk_merged_data,donor_vcf,df_anno)

CPU times: user 5.64 s, sys: 12 ms, total: 5.65 s
Wall time: 5.65 s


In [40]:
df_d

,variants,function,genes,avsnp,GT
0,chr1_14574_A_G,ncRNA_exonic,WASH7P,rs28503599,"[0/0, 0/1, 0/1, 0/0, 0/0, 0/0, 0/1, 0/0, 0/1, ..."
1,chr1_14590_G_A,ncRNA_exonic,WASH7P,rs707679,"[0/0, 0/1, 0/1, 0/0, 0/0, 0/1, 0/1, 0/0, 0/0, ..."
2,chr1_14599_T_A,ncRNA_exonic,WASH7P,rs707680,"[0/1, 0/1, 0/1, 0/0, 0/0, 0/1, 0/1, 0/1, 0/1, ..."
3,chr1_14604_A_G,ncRNA_exonic,WASH7P,rs541940975,"[0/1, 0/1, 0/1, 0/0, 0/0, 0/1, 0/1, 0/1, 0/1, ..."
4,chr1_14610_T_C,ncRNA_exonic,WASH7P,rs878986575,"[0/1, 0/1, 0/1, 0/0, 0/0, 0/1, 0/1, 0/1, 0/1, ..."
...,...,...,...,...,...
128180,chrX_155999818_C_T,intronic,IL9R,rs3093470,"[1/1, 0/0, 1/1, 1/1, 0/1, 0/1, 0/1, 1/1, 1/1, ..."
128181,chrX_155999869_A_G,intronic,IL9R,rs3091261,"[1/1, 1/1, 1/1, 1/1, 1/1, 1/1, 1/1, 1/1, 1/1, ..."
128182,chrX_155999957_G_C,intronic,IL9R,rs3093472,"[1/1, 0/0, 1/1, 1/1, 0/1, 0/1, 0/1, 1/1, 1/1, ..."
128183,chrX_156025297_G_A,downstream,DDX11L16,rs185932868,"[0/1, 0/0, 0/1, 0/0, 0/1, 0/0, 0/0, 0/1, 0/1, ..."


In [37]:
df_b,df_d,m_GT=removedfault(df_b,df_d,m_GT)

In [38]:
m_GT.shape

(127155, 10, 3)

In [20]:
gene_unique=np.unique(df_d["genes"].tolist())

In [21]:
a=np.array(df_b["AD"])
d=np.array(df_b["DP"])
a=np.array(list(map(float,a)))
d=np.array(list(map(float,d)))

In [22]:
model =vireoSNP.VireoBulk(10)

In [23]:
%%time
model.fit(a,d,m_GT,learn_theta=True )

In [24]:
model.theta

array([0.01054732, 0.44960107, 0.99271081])

In [25]:
donor_vcf['samples']

['18-G-017_R1',
 '18-G-018_R1',
 '18-G-019_R1',
 '18-G-020_R1',
 '18-G-021_R1',
 '18-G-022_R1',
 '18-G-023_R1',
 '18-G-024_R1',
 '18-G-025_R1',
 '18-G-026_R1']

In [26]:
premodel=model.psi

In [27]:
premodel

array([0.05715795, 0.04111812, 0.06364389, 0.09476673, 0.14070468,
       0.07099676, 0.05302997, 0.11330121, 0.28828329, 0.07699739])

In [28]:
%%time
bulk_demuti_result= prediction_bulk(df_b,df_d,m_GT,donor_vcf,premodel,gene_unique)

CPU times: user 6min 7s, sys: 23.9 ms, total: 6min 7s
Wall time: 6min 5s


In [29]:
bulk_demuti_result

,18-G-017_R1,18-G-018_R1,18-G-019_R1,18-G-020_R1,18-G-021_R1,18-G-022_R1,18-G-023_R1,18-G-024_R1,18-G-025_R1,18-G-026_R1,chi,p,numSNP
A1BG,0.026748,0.062549,0.102357,0.022211,0.450597,0.027805,0.025727,0.101977,0.163915,0.016113,3.248674,0.953614,3
A1BG-AS1,0.061176,0.010409,0.001743,0.049935,0.568557,0.097229,0.006311,0.011423,0.005862,0.187353,2.903783,0.967988,2
A2M,0.124275,0.001151,0.51316,0.09343,0.001208,0.008507,0.005611,0.113512,0.116549,0.022598,6.817806,0.656082,2
A2M;PZP,0.170696,0.003699,0.326753,0.05438,0.042315,0.016654,0.085925,0.183231,0.113208,0.003139,5.912048,0.748688,1
A2MP1,0.059267,0.157766,0.088763,0.342016,0.023697,0.077471,0.190773,0.041997,0.018249,0.000001,22.83156,0.006586,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,0.212285,0.038627,0.028323,0.213397,0.070399,0.005996,0.136235,0.143128,0.089694,0.061916,2.144412,0.988972,13
ZYG11B,0.03371,0.019217,0.103537,0.342751,0.120287,0.01292,0.01835,0.13429,0.046355,0.168583,10.483539,0.312775,9
ZYX,0.026706,0.009975,0.145311,0.043494,0.113605,0.078055,0.0417,0.003031,0.406706,0.131415,5.698745,0.769649,2
ZZEF1,0.012618,0.059708,0.149221,0.054568,0.227686,0.048068,0.067843,0.118152,0.161034,0.101103,1.681484,0.995563,29


In [ ]:
bulk_demuti_result.to_csv("/media/wgs/subset_stimulation_bam/pbmc10donor/annotated_expo_umi2.csv")